In [1]:
import { requireCytoscape, requireCarbon } from "./lib/draw";

requireCytoscape();
requireCarbon();

# Closures and First-Class Functions 

## Where Were We?

1. **Language primitives** (i.e., building blocks of languages)
    * Last time: first-class functions and anonymous functions
    * This time: **closures**
2. Language paradigms (i.e., combinations of language primitives)
3. Building a language (i.e., designing your own language)

## Goal

1. Learn about **closures** and how they capture the idea of state in first-class functions.
2. Learn about **scope**.
3. One important application of closures is **callbacks**.

## Outline

- Closures by example.
- What exactly is a closure: environment dictionary + code.
- Using closures.

## Closures by Example

#### Example 1

A function whose parameter is the same as another variable in scope.

In [2]:
const x = 1;
function f(x: number): number {
    return x + 1;
}

In [3]:
f(2) // Question: does this return 3 or 2?

3


- The function parameter refers to the closest variable in the text. This is called **lexical scoping**.
- When two variables are named the same, we say that the closet variable in the text **shadows** the further variable.

In [4]:
// The quivalent code
const x1 = 1;
function f(x2: number): number {
    return x2 + 1;
}

#### Example 2

A function within a function with the same variable names.

In [5]:
const x = 1;
function f(x: number): number {
    function g(x: number): number {
        return x + 2;
    }
    return g(x + 1);
}

In [6]:
f(3) // Question: what does this return?

6


In [7]:
// The equivalent code
const x1 = 1
function f(x2: number): number {
    function g(x3: number): number {
        return x3 + 2;
    }
    return g(x2 + 1);
}

#### Example 3

Returning a function with the same variable names.

In [8]:
const x = 1;
function f(x: number): (x: number) => number {
    return (x: number) => {
        return x + 2;
    };
}

In [9]:
f(2);

[Function (anonymous)]


In [10]:
f(1)(3) // Question: what does this return?

5


In [11]:
// The equivalent code
const x1 = 1;
function f(x2: number): (x4: number) => number {
    return (x3: number) => x3 + 2;
}

#### Example 4

Defining an inner function with the same parameter names.

In [12]:
const y = 3;
function f(x: number): number {
    // const g = (x: number, y: number) => x + y;
    function g(x: number, y: number) {
        return x + y;
    }
    return g(x, 1);
}

In [13]:
f(1) // Question: what does this return?

2


In [14]:
// The equivalent code
const y1 = 3;
function f(x1: number): number {
    function g(x2: number, y2: number) {
        return x2 + y2;
    }
    return g(x1, 1);
}

#### Example 5

Our first example of a **closure**.

In [15]:
const y = 3;
function f(x: number): number {
    return x + y; // Question: what does this y refer to? Is it undefined?
}

In [16]:
f(2) // Question: what does this return?

5


In [17]:
// Equivalent code
const y = 3;
// Create an environment that saves all the variables referred to in f's body
const env: { [id: string]: any } = { 
    "y": y
};
function f(env: { [id: string]: any }, x: number): number {
    return x + env["y"];
}
f(env, 2)

5


- The function `f` is said to **capture** the variable `y`. That functions can capture variables should remind you of **references**. It let's you get a handle on things in scope.

A **closure**:
1. Is some code, i.e., a function
2. And an environment, i.e., a dictionary that tells the function how to interpret all the variables that are not in it's local scope.

#### Example 6

Another closure.

In [18]:
const y = 3;
const z = [1, 2];
function f(x: number): number {
    return x + y + z[0] + z[1];
}

In [19]:
f(1) // Question: what does this return?

7


In [20]:
// Equivalent code
const y = 3;
const z = [1, 2];
// Create an environment that saves all the variables referred to in f's body
const env: { [id: string]: any } = { 
    "y": y,
    "z": z 
};
function f(env: { [id: string]: any }, x: number): number {
    return x + env["y"] + env["z"][0] + env["z"][1];
}
f(env, 1)

7


#### Example 7

A closure within a closure.

In [21]:
const y = 3;
const z = [1, 2];
function f(x: number): (x: number) => number {
    const z = [3, 4];
    function g(x: number): number {
        return x + y + z[0] + z[1];
    }
    return g;
}

In [22]:
f(1)(2)  // Question: what does this return?

12


In [23]:
// Equivalent code
const y = 3;
const z1 = [1, 2]
const env1 = { "y": y };
function f(_env1: { [id: string]: any }, x1: number): (x: number) => number {
    const z2 = [3, 4];
    const env2 = { "z2": z2 };
    function g(_env2: { [id: string]: any }, x: number): number {
        return x + _env1["y"] + _env2["z2"][0] + _env2["z2"][1];
    }
    return (x: number) => g(env2, x);
}

In [24]:
f(env1, 1)(2)

12


#### Example 8

Does any of this change if we move from `const` to `let`?

In [25]:
let y = 1;
function f(): (x: number) => number {
    let y = 2;
    function g(x: number): number {
        return x + y;
    }
    return g;
}

console.log(f()(1));
y = 5;
console.log(f()(1));
y = 6;
console.log(f()(1));

3
3
3


#### Example 9

Does any of this change if we move from `const` to `let`?

In [26]:
let y = 1;
function f(): (x: number) => number {
    let y = 2;
    function g(x: number): number {
        return x + y;
    }
    y = 4; // Changing y here
    return g;
}

console.log(f()(1));
y = 5;
console.log(f()(1));
y = 6;
console.log(f()(1));

5
5
5


- The function `g` captures the variable `y = 2`.
- When it changed to `y = 4`, the function g still has access to `y`.

#### Example 10


In [27]:
const arr = [];
for (let i = 0; i < 4; i++) {
    arr.push((x: number) => x + i);
}

In [28]:
// What is printed?
console.log(arr[0](1));
console.log(arr[1](1));
console.log(arr[2](1));
console.log(arr[3](1));

1
2
3
4


#### Example 11

In [29]:
const arr = [];
let i = 0; // I pulled this outside of the loop
for (i = 0; i < 4; i++) {
    arr.push((x: number) => x + i);
}

In [30]:
// What is printed?
console.log(arr[0](1));
console.log(arr[1](1));
console.log(arr[2](1));
console.log(arr[3](1));

5
5
5
5


#### Example 12

In [31]:
const range = [0, 1, 2, 3];
const arr = [];
for (const y of range) {
    arr.push((x: number) => x + y);
}

In [32]:
// What is printed?
console.log(arr[0](1));
console.log(arr[1](1));
console.log(arr[2](1));
console.log(arr[3](1));

1
2
3
4


#### Example 13

Var vs. let

In [33]:
const arr = [];
for (var i = 0; i < 4; i++) {
    arr.push((x: number) => x + i);
}

In [34]:
console.log(arr[0](1));
console.log(arr[1](1));
console.log(arr[2](1));
console.log(arr[3](1));

5
5
5
5


#### DO NOT USE VAR!

## What's the big deal with Closures?

### Application 1: Closures and Encapsulation

- Recall that one of the benefits of Objects and Classes was that we could hide data.

In [35]:
class Counter {
    private count: number;
    
    constructor() {
        this.count = 0;
    }
    
    increment() {
        this.count += 1;
        return this.count;
    }
    
    getCount() {
        return this.count;
    }
}

In [36]:
const counter = new Counter();
console.log(counter.increment());
console.log(counter.increment());
console.log(counter.increment());

1
2
3


In [37]:
// This fails at compile-time because count is private
try {
    counter.count += 1;
} catch (e) {
    console.log(e);
}

3:13 - Property 'count' is private and only accessible within class 'Counter'.


In [38]:
console.log(counter.increment());

4


#### Attempt with functions: Take 1

In [39]:
let count = 0;

function increment() {
    count += 1;
    return count;
}

console.log(increment());
console.log(increment());
console.log(increment());
count += 1;  // bad behavior, want to disallow
console.log(increment());

1
2
3
5


- The issue is that count is still in scope. We could not hide the variable count.
- What if we wanted two counters?

#### Attempt with functions: Take 2

In [40]:
type FunCounter = {
    increment: () => void,
    getCount: () => number
};

function makeCounter(): FunCounter {
    let count = 0;
    
    function _increment() {
        // Increment **captures** the variable count
        count += 1;
        return count;
    }
    
    function _getCount() {
        return count;
    }
    
    return {
        increment: _increment,
        getCount: _getCount
    };
}

In [41]:
const counter = makeCounter();
console.log(counter.increment());
console.log(counter.increment());
console.log(counter.increment());

1
2
3


In [42]:
// This also fails at compile-time because count is private
try {
    counter.count += 1;
} catch (e) {
    console.log(e);
}

3:13 - Property 'count' does not exist on type 'FunCounter'.


In [ ]:
console.log(counter.increment());
console.log(counter.getCount());

#### Closure's are kind of like "Objects"

- We just saw how to encode objects with first-class functions.
- Crucially, we needed the ability to close over variables in a function's local scope.
- Challenge: how would we encode something like inheritance?

### Application 2: Closures and Callbacks

- Closures are great for implementing **callbacks**: call this function when some event happens.
- Events are commonly things such as user-input (e.g., clicks, typing).
- We are in TypeScript, which is a superset of JavaScript, so we should have access to user-events from the browser.
- Since we are in a Jupyter notebook, we will use our Jupyter notebook's ability to display raw HTML to illustrate this concept first.

In [43]:
import * as tslab from "tslab";

In [44]:
tslab.display.html(`
<!-- This part is HTML -->
<p onclick="callback(this)">Click me to change my text color.</p>

<!-- This part is TypeScript -->
<script>
let lastIdx = 0;
const colors = ['red', 'green', 'blue'];

function callback(elmnt) {
    // This HTML element has a style.color property
    // colors refers to the on in lexical scope and will be captured by callback.
    // lastIdx refers to the one in lexical scope and will be captured by callback.
    elmnt.style.color = colors[lastIdx];

    lastIdx = (lastIdx + 1) % (colors.length);
}
</script>
`);

<!-- This part is HTML -->
 Click me to change my text color. 

<!-- This part is TypeScript -->

- Pretty cool right?
- But ... we're programming with strings again. This should remind you of all the bad things with copy-and-paste.
    * No syntax highlighting
    * No static checking
    * What if I can't modify the code?
- How can we fix this?

#### Closures to the rescue!

In [45]:
// Let's just package our code inside a function.
function codeBlock() {
    let lastIdx = 0;
    const colors = ['red', 'green', 'blue'];

    function callback(elmnt) {
        // This HTML element has a style.color property
        // colors refers to the on in lexical scope and will be captured by callback.
        // lastIdx refers to the one in lexical scope and will be captured by callback.
        elmnt.style.color = colors[lastIdx];
        lastIdx = (lastIdx + 1) % (colors.length);
    }
    
    return callback;
}

In [46]:
function displayHTMLWithCallback(closure) {
    tslab.display.html(`
    <p onclick="callback(this)">Click me to change my text color.</p>

    <script>
    // We won't understand this fully for now, but we are essentially using code to do the copy-paste for us.
    ${closure.toString()}
    // Calling our closure will unpackage the function.
    const callback = ${closure.name}();
    </script>
    `);    
}

displayHTMLWithCallback(codeBlock)

Click me to change my text color.

### Application 3: Closures and Pure Functions

In [47]:
function pureAddArray(arr1: number[], arr2: number[]): number[] {
    const copy = arr1.map((x) => x);  // Question: what does this code do?

    function inplaceAdd(acc: number[]): void {
        for (let i = 0; i < acc.length; i++) {
            acc[i] += arr2[i];
        }
    }

    inplaceAdd(copy);
    return copy;
}

In [48]:
const arr = [1, 2, 3];
console.log(arr);
console.log(pureAddArray(arr, arr));
console.log(arr);

[ 1, 2, 3 ]
[ 2, 4, 6 ]
[ 1, 2, 3 ]


## Summary

- Today we saw closures, i.e., first-class functions + dictionaries (i.e., references / state).
- We also took a deeper look at scoping.
- Applications of closures include callbacks and encapsulation. 